In [1]:
import pandas as pd
from OptionPlayTranslator import *

In [2]:

srcDataFrame = pd.read_csv('optionsDX/aapl_2022/aapl_eod_2022q4-86egbl/aapl_eod_202210.txt')
# print(srcDataFrame.columns)
df = pd.DataFrame(srcDataFrame)
df.columns = df.columns.str.strip()
# print(df.columns)




In [18]:
quote_date = df.loc[df['[QUOTE_DATE]'] == '2022-10-03']

# print(quote_date.head())

quote_date.to_dict('records')
print(quote_date)


     [QUOTE_UNIXTIME]   [QUOTE_READTIME] [QUOTE_DATE]  [QUOTE_TIME_HOURS]  \
0          1664827200   2022-10-03 16:00   2022-10-03                16.0   
1          1664827200   2022-10-03 16:00   2022-10-03                16.0   
2          1664827200   2022-10-03 16:00   2022-10-03                16.0   
3          1664827200   2022-10-03 16:00   2022-10-03                16.0   
4          1664827200   2022-10-03 16:00   2022-10-03                16.0   
..                ...                ...          ...                 ...   
880        1664827200   2022-10-03 16:00   2022-10-03                16.0   
881        1664827200   2022-10-03 16:00   2022-10-03                16.0   
882        1664827200   2022-10-03 16:00   2022-10-03                16.0   
883        1664827200   2022-10-03 16:00   2022-10-03                16.0   
884        1664827200   2022-10-03 16:00   2022-10-03                16.0   

     [UNDERLYING_LAST] [EXPIRE_DATE]  [EXPIRE_UNIX]   [DTE]  [C_DELTA]  \
0

In [16]:
pred_chain = quote_date.loc[quote_date['[EXPIRE_DATE]'] == '2022-10-14']
pred_chain.head(10)

,[QUOTE_UNIXTIME],[QUOTE_READTIME],[QUOTE_DATE],[QUOTE_TIME_HOURS],[UNDERLYING_LAST],[EXPIRE_DATE],[EXPIRE_UNIX],[DTE],[C_DELTA],[C_GAMMA],...,[P_LAST],[P_DELTA],[P_GAMMA],[P_VEGA],[P_THETA],[P_RHO],[P_IV],[P_VOLUME],[STRIKE_DISTANCE],[STRIKE_DISTANCE_PCT]


In [3]:
df['[QUOTE_DATE]'] = df.get('[QUOTE_DATE]').str.strip()
# df['[QUOTE_DATE]']

In [4]:
dayGroups = df.groupby('[QUOTE_DATE]')
# print(dayGroups.groups.keys())
# print(dayGroups.get_group('2022-10-03'))




In [5]:
print(type( dayGroups))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


In [6]:
OPT = OptionPlayTranslator('AAPL', dayGroups)
OPT.findPlays('2022-10-03', '2022-10-15')



 COLUMNS!! 
 Index(['[QUOTE_UNIXTIME]', '[QUOTE_READTIME]', '[QUOTE_DATE]',
       '[QUOTE_TIME_HOURS]', '[UNDERLYING_LAST]', '[EXPIRE_DATE]',
       '[EXPIRE_UNIX]', '[DTE]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]',
       '[C_THETA]', '[C_RHO]', '[C_IV]', '[C_VOLUME]', '[C_LAST]', '[C_SIZE]',
       '[C_BID]', '[C_ASK]', '[STRIKE]', '[P_BID]', '[P_ASK]', '[P_SIZE]',
       '[P_LAST]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]',
       '[P_RHO]', '[P_IV]', '[P_VOLUME]', '[STRIKE_DISTANCE]',
       '[STRIKE_DISTANCE_PCT]'],
      dtype='object')
nearest_expiry  2022-10-14
Empty DataFrame
Columns: [[QUOTE_UNIXTIME], [QUOTE_READTIME], [QUOTE_DATE], [QUOTE_TIME_HOURS], [UNDERLYING_LAST], [EXPIRE_DATE], [EXPIRE_UNIX], [DTE], [C_DELTA], [C_GAMMA], [C_VEGA], [C_THETA], [C_RHO], [C_IV], [C_VOLUME], [C_LAST], [C_SIZE], [C_BID], [C_ASK], [STRIKE], [P_BID], [P_ASK], [P_SIZE], [P_LAST], [P_DELTA], [P_GAMMA], [P_VEGA], [P_THETA], [P_RHO], [P_IV], [P_VOLUME], [STRIKE_DISTANCE], [STRIKE_DISTANC

In [7]:
import datetime
from datetime import date, timedelta
import numpy as np

Given a price prediction on a current day, you look at an option chain seeking the most efficient way to express the prediction.

Given that the prediction comes from a black box, we want to look for contracts expiring as close to the prediction date as possible. The prediction doesn't say anything about what happens between the dates.

Contracts all expire on Fridays

For now, we'll examine only the contracts closest to the prediction date that expire before the contract expires. The exception is a prediction for the same week, we'll increment to Friday of that week, since those are the closest vehicles for expressing the prediction.

However, the option-play-translations for all predictions for a closing price on a day other than Friday will suffer, potentially greatly, from the mismatch of the dates (with Thursday suffering the most).

Instead of limiting functionality, I propose that we just evaluate predictions for any market day other than the Friday separately.

We could also round Wednesday/ Thursday up and Monday/ Tuesday down, but the validity will be suspect regardless.

In [8]:
curDate = np.array('2023-05-30'.strip().split('-'), dtype=int)
futDate = np.array('2023-06-10'.strip().split('-'), dtype=int)
cur = date(*curDate)
fut = date(*futDate)

# date.weekday() moday = 0, sunday = 6, friday = 4
print('cur is', cur, ', cur weekday', cur.weekday())
if ((fut - cur).days < 5 ):
    # same week, increment to friday
    while  fut.weekday() != 4:
        fut += timedelta(1)
else:
    # get first friday before predicted date
    while fut.weekday() != 4:
        fut -= timedelta(1)
print('fut is ', fut, ', weekday ', fut.weekday())
assert(fut > cur)



cur is 2023-05-30 , cur weekday 1
fut is  2023-06-09 , weekday  4
